In [158]:
import pyodbc

server = 'localhost'
database = 'AdventureWorks'

conn = pyodbc.connect(f'DRIVER=ODBC Driver 18 for SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes;Encrypt=no;')

cursor = conn.cursor()
cursor.execute('SELECT @@VERSION')
row = cursor.fetchone()
print(row)


('Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) \n\tOct  8 2022 05:58:25 \n\tCopyright (C) 2022 Microsoft Corporation\n\tEnterprise Evaluation Edition (64-bit) on Windows 10 Home Single Language 10.0 <X64> (Build 26100: ) (Hypervisor)\n',)


In [159]:
cursor.execute("SELECT COUNT(*) AS TableCount FROM information_schema.tables WHERE table_type = 'BASE TABLE';")
row = cursor.fetchone()
print(row)

(31,)


In [160]:
conn_str = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER=localhost;DATABASE=AdventureWorks;'

In [161]:
from langchain_groq import ChatGroq
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase

In [162]:
uri = 'mssql+pyodbc://localhost/TPMS?driver=ODBC+Driver+18+for+SQL+Server&TrustServerCertificate=yes&Encrypt=no'
db = SQLDatabase.from_uri(uri)

llm = ChatGroq(model="llama-3.1-8b-instant", api_key="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")

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [163]:
db.get_usable_table_names()

['pc_work_group',
 'pr_ADEO_TaskMana_Data_Delivera',
 'pr_ADEO_TaskMana_Data_Mileston',
 'pr_ADEO_TaskMana_Data_RiskRegi',
 'pr_Data_Comments']

In [164]:
db_info = db.get_table_info()

In [165]:
query = "What are all the comments updated in the Gamma project"

In [166]:
prompt = f"""
You are a MS SQL server expert, Your task is to create the SQL query for the users question using the given tables schema
Database tabels schema:
{db_info}
User's query: {query}
Return only SQL query without any explanations and do not keep ```sql in the response
"""

In [167]:
prompt

"\nYou are a MS SQL server expert, Your task is to create the SQL query for the users question using the given tables schema\nDatabase tabels schema:\n\nCREATE TABLE [pr_ADEO_TaskMana_Data_Delivera] (\n\t[ID] VARCHAR(256) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, \n\t[Name] VARCHAR(256) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, \n\t[MilestoneID] VARCHAR(256) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, \n\t[StartDateTime] DATETIME NULL, \n\t[EndDateTime] DATETIME NULL, \n\t[pySelected] VARCHAR(5) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, \n\t[Status] VARCHAR(256) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, \n\tCONSTRAINT [PK__pr_ADEO___3214EC279CF9DC25] PRIMARY KEY ([ID]), \n\tCONSTRAINT [FK__pr_ADEO_T__Miles__4222D4EF] FOREIGN KEY([MilestoneID]) REFERENCES [pr_ADEO_TaskMana_Data_Mileston] ([ID])\n)\n\n/*\n3 rows from pr_ADEO_TaskMana_Data_Delivera table:\nID\tName\tMilestoneID\tStartDateTime\tEndDateTime\tpySelected\tStatus\nD001\tDeliverable 1\tM001\t2023-01-20 09:00:00\t2023-02-

In [172]:
response = llm.invoke(prompt)

In [173]:
print(response.content)

SELECT p.pzInsKey, c.pyDescription 
FROM pc_work_group p 
INNER JOIN pr_Data_Comments c ON p.pzInsKey = c.pxRefObjectKey 
WHERE p.pyLabel = 'Project Gamma'


In [156]:
query_response = db.run(response.content, include_columns=True)

In [157]:
query_response

"[{'pyDescription': 'Compatibility concerns logged.'}]"

In [126]:
response_prompt = f""" 
You are AI expert tasked for create a natural response message from the SQL query response for the users question.
The response must be concise
The SQL query response is: {query_response}
The user question is: {query}
"""

In [127]:
message = llm.invoke(response_prompt).content
print(message)

Here's a concise natural response based on the SQL query response and the user's question:

"Compatibility concerns have been logged in the Gamma project.


In [ ]:
1. Project updates, comments table
2. Status of the project
3. Base 64 to img and pdf
4. Project category 
5. Project closing comments and details

SyntaxError: invalid syntax (1657033885.py, line 1)

In [ ]:
import base64
import re

def is_base64(string):
    """
    Checks if a string is a valid Base64-encoded string.

    Parameters:
        string (str): The string to validate.

    Returns:
        bool: True if the string is valid Base64, False otherwise.
    """
    base64_pattern = re.compile(r'^[A-Za-z0-9+/=\s]+$')
    return base64_pattern.fullmatch(string) is not None

def extract_text_from_base64(base64_file_path, output_file_path=None):
    try:
        with open(base64_file_path, 'r') as file:
            base64_data = file.read()

        if not is_base64(base64_data.strip()):
            print("Error: The file does not contain valid Base64-encoded text.")
            return None

        decoded_data = base64.b64decode(base64_data)

        try:
            decoded_text = decoded_data.decode('utf-8')
            if output_file_path:
                with open(output_file_path, 'w') as output_file:
                    output_file.write(decoded_text)
            return decoded_text
        except UnicodeDecodeError:
            if output_file_path:
                with open(output_file_path, 'wb') as output_file:
                    output_file.write(decoded_data)
            print("Warning: Decoded data is binary and has been saved to the output file.")
            return None

    except FileNotFoundError:
        print(f"Error: The file '{base64_file_path}' does not exist.")
    except (base64.binascii.Error, ValueError) as e:
        print(f"Error: Failed to decode Base64 content. Details: {e}")

# Example usage
if __name__ == "__main__":
    input_file = "base64_input.txt"
    output_file = "decoded_output.png" 

    text = extract_text_from_base64(input_file, output_file)
    if text:
        print("Decoded text:")
        print(text)
